# LLM 微調技術 (LLM Fine-tuning Techniques)

**對應課程**: 李宏毅 2025 Spring ML HW5, 2025 Fall GenAI-ML HW7

本 notebook 涵蓋大型語言模型的微調技術，包括 Full Fine-tuning、LoRA、QLoRA 等高效微調方法。

## 學習目標
1. 理解 Full Fine-tuning vs PEFT 的差異
2. 掌握 LoRA 的原理與實作
3. 學會 QLoRA（4-bit 量化 + LoRA）
4. 了解訓練資料格式與準備
5. 實作使用 PEFT 庫微調小型 LLM

## Part 1: 為什麼需要微調？

### 1.1 預訓練 vs 微調

```
┌─────────────────────────────────────────────────────────────────┐
│                     LLM 訓練階段                                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌──────────────────────────────────────────────────────────┐  │
│  │                    Pre-training                           │  │
│  │  • 海量無標註資料（TB 級別）                               │  │
│  │  • 自監督學習（Next Token Prediction）                    │  │
│  │  • 學習語言的通用表示                                     │  │
│  │  • 需要大量 GPU 資源（數千 GPU-days）                     │  │
│  └──────────────────────────────────────────────────────────┘  │
│                            ↓                                    │
│  ┌──────────────────────────────────────────────────────────┐  │
│  │                    Fine-tuning                            │  │
│  │  • 小量標註資料（數千到數萬筆）                            │  │
│  │  • 監督學習（特定任務）                                   │  │
│  │  • 適應特定領域或任務                                     │  │
│  │  • 較少 GPU 資源（單卡或數卡）                            │  │
│  └──────────────────────────────────────────────────────────┘  │
│                            ↓                                    │
│  ┌──────────────────────────────────────────────────────────┐  │
│  │                    Alignment (RLHF/DPO)                   │  │
│  │  • 人類偏好資料                                           │  │
│  │  • 讓模型輸出符合人類期望                                  │  │
│  └──────────────────────────────────────────────────────────┘  │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### 1.2 微調的應用場景

```
┌────────────────────────────────────────────────────────────────┐
│                   微調的主要用途                                │
├────────────────────────────────────────────────────────────────┤
│                                                                │
│  1. 領域適應 (Domain Adaptation)                               │
│     • 醫療、法律、金融等專業領域                               │
│     • 學習領域專有術語和知識                                   │
│                                                                │
│  2. 任務特化 (Task Specialization)                             │
│     • 程式碼生成、摘要、翻譯                                   │
│     • 提高特定任務的效能                                       │
│                                                                │
│  3. 風格調整 (Style Adaptation)                                │
│     • 品牌語調、正式/非正式                                    │
│     • 特定角色或人格                                           │
│                                                                │
│  4. 指令遵循 (Instruction Following)                           │
│     • 讓模型更好地遵循指令                                     │
│     • 提高對話能力                                             │
│                                                                │
└────────────────────────────────────────────────────────────────┘
```

In [ ]:
# 環境設置
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("\n建議安裝的套件:")
print("  pip install transformers peft bitsandbytes accelerate datasets")

## Part 2: Full Fine-tuning vs PEFT

### 2.1 Full Fine-tuning 的挑戰

```
┌────────────────────────────────────────────────────────────────┐
│                 Full Fine-tuning 的問題                        │
├────────────────────────────────────────────────────────────────┤
│                                                                │
│  記憶體需求 (以 7B 模型為例):                                   │
│  ┌──────────────────────────────────────────────────────────┐ │
│  │ 模型參數 (FP16):     7B × 2 bytes = 14 GB                │ │
│  │ 梯度 (FP16):         7B × 2 bytes = 14 GB                │ │
│  │ 優化器狀態 (Adam):   7B × 8 bytes = 56 GB                │ │
│  │ 激活值:              視 batch size 而定                   │ │
│  │ ─────────────────────────────────────────────            │ │
│  │ 總計:                約 84 GB + 激活值                    │ │
│  └──────────────────────────────────────────────────────────┘ │
│                                                                │
│  其他問題:                                                     │
│  • 災難性遺忘 (Catastrophic Forgetting)                       │
│  • 每個任務需要儲存完整模型副本                                │
│  • 訓練時間長                                                 │
│                                                                │
└────────────────────────────────────────────────────────────────┘
```

In [ ]:
# 視覺化不同模型規模的記憶體需求
def visualize_memory_requirements():
    """視覺化 Full Fine-tuning 的記憶體需求"""
    
    model_sizes = ['1B', '3B', '7B', '13B', '70B']
    param_counts = [1, 3, 7, 13, 70]  # 十億
    
    # 計算各項記憶體需求 (GB)
    model_memory = [p * 2 for p in param_counts]  # FP16
    gradient_memory = [p * 2 for p in param_counts]  # FP16
    optimizer_memory = [p * 8 for p in param_counts]  # Adam (2 states × 4 bytes each)
    
    x = np.arange(len(model_sizes))
    width = 0.25
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    bars1 = ax.bar(x - width, model_memory, width, label='Model Parameters', color='steelblue')
    bars2 = ax.bar(x, gradient_memory, width, label='Gradients', color='coral')
    bars3 = ax.bar(x + width, optimizer_memory, width, label='Optimizer States (Adam)', color='seagreen')
    
    ax.set_xlabel('Model Size')
    ax.set_ylabel('Memory (GB)')
    ax.set_title('Full Fine-tuning Memory Requirements (FP16)')
    ax.set_xticks(x)
    ax.set_xticklabels(model_sizes)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    # 標註總計
    totals = [m + g + o for m, g, o in zip(model_memory, gradient_memory, optimizer_memory)]
    for i, total in enumerate(totals):
        ax.annotate(f'Total: {total} GB', 
                   xy=(i, optimizer_memory[i]), 
                   xytext=(i, max(totals) * 0.9),
                   ha='center', fontsize=9,
                   arrowprops=dict(arrowstyle='->', color='gray', alpha=0.5))
    
    # 標註常見 GPU VRAM
    gpu_vrams = [('RTX 4090', 24), ('A100-40GB', 40), ('A100-80GB', 80)]
    for name, vram in gpu_vrams:
        ax.axhline(y=vram, color='red', linestyle='--', alpha=0.3)
        ax.text(len(model_sizes) - 0.5, vram + 2, f'{name} ({vram}GB)', fontsize=8, color='red')
    
    plt.tight_layout()
    plt.show()

visualize_memory_requirements()

### 2.2 PEFT (Parameter-Efficient Fine-Tuning)

```
┌────────────────────────────────────────────────────────────────┐
│                     PEFT 方法概覽                               │
├────────────────────────────────────────────────────────────────┤
│                                                                │
│  1. Adapter Methods                                            │
│     • 在 Transformer 層間插入小型網路                          │
│     • 只訓練 adapter 參數                                      │
│                                                                │
│  2. LoRA (Low-Rank Adaptation)                                 │
│     • 用低秩矩陣近似權重更新                                   │
│     • 最流行的 PEFT 方法                                       │
│                                                                │
│  3. Prefix Tuning / Prompt Tuning                              │
│     • 訓練可學習的 prefix tokens                               │
│     • 不修改模型參數                                           │
│                                                                │
│  4. IA3 (Infused Adapter by Inhibiting and Amplifying)         │
│     • 學習縮放向量                                             │
│     • 參數量更少                                               │
│                                                                │
│  比較:                                                         │
│  ┌────────────┬─────────────┬──────────────┬─────────────┐    │
│  │ 方法        │ 可訓練參數   │ 推理開銷      │ 效果        │    │
│  ├────────────┼─────────────┼──────────────┼─────────────┤    │
│  │ Full FT    │ 100%        │ 無           │ 最佳        │    │
│  │ LoRA       │ 0.1-1%      │ 可合併(無)   │ 接近 Full   │    │
│  │ Adapter    │ 1-5%        │ 有           │ 良好        │    │
│  │ Prefix     │ <0.1%       │ 有           │ 中等        │    │
│  └────────────┴─────────────┴──────────────┴─────────────┘    │
│                                                                │
└────────────────────────────────────────────────────────────────┘
```

## Part 3: LoRA 深入解析

### 3.1 LoRA 原理

LoRA 的核心想法：權重更新 $\Delta W$ 具有低秩結構。

$$W' = W + \Delta W = W + BA$$

其中：
- $W \in \mathbb{R}^{d \times k}$: 原始權重（凍結）
- $B \in \mathbb{R}^{d \times r}$: 低秩矩陣
- $A \in \mathbb{R}^{r \times k}$: 低秩矩陣
- $r \ll \min(d, k)$: rank（通常 4-64）

```
┌────────────────────────────────────────────────────────────────┐
│                      LoRA 結構示意                              │
├────────────────────────────────────────────────────────────────┤
│                                                                │
│                        Input x                                 │
│                           │                                    │
│              ┌────────────┼────────────┐                       │
│              │            │            │                       │
│              ▼            │            ▼                       │
│         ┌────────┐       │       ┌────────┐                   │
│         │   W    │       │       │   A    │ r×k               │
│         │ (凍結) │       │       │ (可訓練)│                   │
│         │ d×k    │       │       └────┬───┘                   │
│         └────┬───┘       │            │                       │
│              │           │            ▼                       │
│              │           │       ┌────────┐                   │
│              │           │       │   B    │ d×r               │
│              │           │       │ (可訓練)│                   │
│              │           │       └────┬───┘                   │
│              │           │            │ × (α/r)               │
│              │           │            │                       │
│              └─────────+ │ +──────────┘                       │
│                          │                                    │
│                          ▼                                    │
│                       Output                                   │
│                                                                │
│  參數量: d×k (凍結) + r×k + d×r (可訓練)                       │
│  例如: d=4096, k=4096, r=8                                     │
│       原始: 16M, LoRA: 64K (0.4%)                              │
│                                                                │
└────────────────────────────────────────────────────────────────┘
```

In [ ]:
# LoRA 層的實作
class LoRALayer(nn.Module):
    """LoRA (Low-Rank Adaptation) 層"""
    
    def __init__(self, 
                 in_features: int, 
                 out_features: int, 
                 rank: int = 8, 
                 alpha: float = 16.0,
                 dropout: float = 0.0):
        super().__init__()
        
        self.rank = rank
        self.alpha = alpha
        self.scaling = alpha / rank  # LoRA 縮放因子
        
        # 原始線性層（凍結）
        self.linear = nn.Linear(in_features, out_features, bias=False)
        self.linear.weight.requires_grad = False
        
        # LoRA 矩陣
        self.lora_A = nn.Linear(in_features, rank, bias=False)
        self.lora_B = nn.Linear(rank, out_features, bias=False)
        
        # Dropout
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
        
        # 初始化
        nn.init.kaiming_uniform_(self.lora_A.weight, a=np.sqrt(5))
        nn.init.zeros_(self.lora_B.weight)  # 初始化為 0，確保開始時 LoRA 不改變輸出
    
    def forward(self, x):
        # 原始輸出
        original_output = self.linear(x)
        
        # LoRA 增量
        lora_output = self.lora_B(self.lora_A(self.dropout(x))) * self.scaling
        
        return original_output + lora_output
    
    def merge_weights(self):
        """將 LoRA 權重合併到原始權重（用於推理加速）"""
        with torch.no_grad():
            # W' = W + BA * scaling
            delta_w = (self.lora_B.weight @ self.lora_A.weight) * self.scaling
            self.linear.weight.add_(delta_w)
    
    @property
    def trainable_params(self):
        return sum(p.numel() for p in [self.lora_A.weight, self.lora_B.weight])
    
    @property
    def total_params(self):
        return self.linear.weight.numel() + self.trainable_params

# 測試 LoRA 層
in_features, out_features = 4096, 4096
rank = 8

lora_layer = LoRALayer(in_features, out_features, rank=rank, alpha=16.0)

print(f"輸入維度: {in_features}, 輸出維度: {out_features}, Rank: {rank}")
print(f"原始參數量: {in_features * out_features:,}")
print(f"LoRA 參數量: {lora_layer.trainable_params:,}")
print(f"參數比例: {lora_layer.trainable_params / (in_features * out_features) * 100:.2f}%")

# 測試前向傳播
x = torch.randn(2, 128, in_features)
output = lora_layer(x)
print(f"\n輸入形狀: {x.shape}")
print(f"輸出形狀: {output.shape}")

In [ ]:
# 視覺化 LoRA 參數效率
def visualize_lora_efficiency():
    """視覺化不同 rank 下的 LoRA 參數效率"""
    
    hidden_size = 4096  # 典型的 LLM hidden size
    ranks = [1, 2, 4, 8, 16, 32, 64, 128, 256]
    
    # 計算參數量
    original_params = hidden_size * hidden_size
    lora_params = [(hidden_size * r + r * hidden_size) for r in ranks]
    param_ratios = [lp / original_params * 100 for lp in lora_params]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # 左圖：參數量
    ax1.bar(range(len(ranks)), [original_params] * len(ranks), 
           alpha=0.3, label='Original', color='gray')
    ax1.bar(range(len(ranks)), lora_params, 
           alpha=0.8, label='LoRA Trainable', color='steelblue')
    ax1.set_xlabel('LoRA Rank')
    ax1.set_ylabel('Number of Parameters')
    ax1.set_title('LoRA vs Original Parameters')
    ax1.set_xticks(range(len(ranks)))
    ax1.set_xticklabels(ranks)
    ax1.legend()
    ax1.set_yscale('log')
    ax1.grid(axis='y', alpha=0.3)
    
    # 右圖：參數比例
    bars = ax2.bar(range(len(ranks)), param_ratios, color='coral')
    ax2.set_xlabel('LoRA Rank')
    ax2.set_ylabel('Trainable Parameter Ratio (%)')
    ax2.set_title('LoRA Parameter Efficiency')
    ax2.set_xticks(range(len(ranks)))
    ax2.set_xticklabels(ranks)
    ax2.grid(axis='y', alpha=0.3)
    
    # 標註常用 rank
    for i, (r, ratio) in enumerate(zip(ranks, param_ratios)):
        if r in [8, 16, 32]:
            ax2.annotate(f'{ratio:.2f}%', xy=(i, ratio), 
                        xytext=(i, ratio + 0.5),
                        ha='center', fontsize=9, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\n典型 rank 設定:")
    print(f"  rank=8:  {param_ratios[3]:.3f}% of parameters")
    print(f"  rank=16: {param_ratios[4]:.3f}% of parameters")
    print(f"  rank=32: {param_ratios[5]:.3f}% of parameters")

visualize_lora_efficiency()

### 3.2 LoRA 應用到 Transformer

通常對以下層應用 LoRA：
- Query (Q) 投影
- Key (K) 投影
- Value (V) 投影
- Output (O) 投影
- FFN 層（可選）

In [ ]:
class LoRAAttention(nn.Module):
    """帶 LoRA 的 Multi-Head Attention"""
    
    def __init__(self, 
                 hidden_size: int = 768,
                 num_heads: int = 12,
                 lora_rank: int = 8,
                 lora_alpha: float = 16.0,
                 lora_dropout: float = 0.0,
                 target_modules: List[str] = ['q', 'v']):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads
        
        # 原始投影層
        self.q_proj = nn.Linear(hidden_size, hidden_size, bias=False)
        self.k_proj = nn.Linear(hidden_size, hidden_size, bias=False)
        self.v_proj = nn.Linear(hidden_size, hidden_size, bias=False)
        self.o_proj = nn.Linear(hidden_size, hidden_size, bias=False)
        
        # 凍結原始權重
        for proj in [self.q_proj, self.k_proj, self.v_proj, self.o_proj]:
            for param in proj.parameters():
                param.requires_grad = False
        
        # LoRA 層
        self.lora_modules = {}
        for name in target_modules:
            self.lora_modules[name] = {
                'A': nn.Linear(hidden_size, lora_rank, bias=False),
                'B': nn.Linear(lora_rank, hidden_size, bias=False)
            }
            # 初始化
            nn.init.kaiming_uniform_(self.lora_modules[name]['A'].weight)
            nn.init.zeros_(self.lora_modules[name]['B'].weight)
            
            # 註冊為子模組
            self.add_module(f'lora_{name}_A', self.lora_modules[name]['A'])
            self.add_module(f'lora_{name}_B', self.lora_modules[name]['B'])
        
        self.scaling = lora_alpha / lora_rank
        self.dropout = nn.Dropout(lora_dropout)
    
    def _apply_lora(self, x, proj, lora_name):
        """應用 LoRA 到投影層"""
        output = proj(x)
        
        if lora_name in self.lora_modules:
            lora_A = self.lora_modules[lora_name]['A']
            lora_B = self.lora_modules[lora_name]['B']
            lora_output = lora_B(lora_A(self.dropout(x))) * self.scaling
            output = output + lora_output
        
        return output
    
    def forward(self, hidden_states, attention_mask=None):
        batch_size, seq_len, _ = hidden_states.shape
        
        # 投影（帶 LoRA）
        q = self._apply_lora(hidden_states, self.q_proj, 'q')
        k = self._apply_lora(hidden_states, self.k_proj, 'k')
        v = self._apply_lora(hidden_states, self.v_proj, 'v')
        
        # 重塑為多頭
        q = q.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        # 注意力計算
        attn_weights = torch.matmul(q, k.transpose(-2, -1)) / np.sqrt(self.head_dim)
        
        if attention_mask is not None:
            attn_weights = attn_weights + attention_mask
        
        attn_weights = F.softmax(attn_weights, dim=-1)
        attn_output = torch.matmul(attn_weights, v)
        
        # 合併頭
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.hidden_size)
        
        # 輸出投影
        output = self._apply_lora(attn_output, self.o_proj, 'o')
        
        return output

# 測試
lora_attn = LoRAAttention(hidden_size=768, num_heads=12, lora_rank=8, target_modules=['q', 'v'])

# 計算參數量
total_params = sum(p.numel() for p in lora_attn.parameters())
trainable_params = sum(p.numel() for p in lora_attn.parameters() if p.requires_grad)

print(f"總參數量: {total_params:,}")
print(f"可訓練參數量: {trainable_params:,}")
print(f"可訓練比例: {trainable_params / total_params * 100:.2f}%")

## Part 4: QLoRA（量化 + LoRA）

### 4.1 QLoRA 原理

QLoRA 結合 4-bit 量化和 LoRA，進一步降低記憶體需求。

```
┌────────────────────────────────────────────────────────────────┐
│                      QLoRA 技術棧                               │
├────────────────────────────────────────────────────────────────┤
│                                                                │
│  1. 4-bit NormalFloat (NF4) 量化                              │
│     • 專為正態分布權重設計的量化格式                           │
│     • 比 INT4 有更好的精度                                     │
│                                                                │
│  2. Double Quantization                                        │
│     • 量化「量化常數」以進一步節省記憶體                       │
│     • 每 64 個參數的量化常數再量化為 FP8                       │
│                                                                │
│  3. Paged Optimizers                                          │
│     • 使用 NVIDIA unified memory                              │
│     • 自動處理記憶體溢出                                      │
│                                                                │
│  記憶體比較 (7B 模型):                                         │
│  ┌───────────────────────────────────────────────────────┐   │
│  │ Full FT (FP16):     ~84 GB                             │   │
│  │ LoRA (FP16):        ~14 GB (模型) + 少量可訓練參數      │   │
│  │ QLoRA (NF4):        ~4 GB (模型) + 少量可訓練參數       │   │
│  └───────────────────────────────────────────────────────┘   │
│                                                                │
└────────────────────────────────────────────────────────────────┘
```

In [ ]:
# 視覺化 QLoRA 記憶體節省
def visualize_qlora_memory():
    """視覺化 QLoRA 的記憶體節省效果"""
    
    model_sizes = ['1B', '3B', '7B', '13B', '30B']
    param_counts = [1, 3, 7, 13, 30]  # 十億
    
    # 計算記憶體需求 (GB)
    full_ft_memory = [p * 12 for p in param_counts]  # 模型 + 梯度 + 優化器
    lora_fp16 = [p * 2 for p in param_counts]  # 只有凍結模型
    qlora_nf4 = [p * 0.5 for p in param_counts]  # 4-bit 量化
    
    x = np.arange(len(model_sizes))
    width = 0.25
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    bars1 = ax.bar(x - width, full_ft_memory, width, label='Full Fine-tuning (FP16)', color='coral')
    bars2 = ax.bar(x, lora_fp16, width, label='LoRA (FP16 frozen)', color='steelblue')
    bars3 = ax.bar(x + width, qlora_nf4, width, label='QLoRA (NF4)', color='seagreen')
    
    ax.set_xlabel('Model Size')
    ax.set_ylabel('Memory (GB)')
    ax.set_title('Memory Requirements: Full FT vs LoRA vs QLoRA')
    ax.set_xticks(x)
    ax.set_xticklabels(model_sizes)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    # 標註 GPU VRAM 線
    ax.axhline(y=16, color='red', linestyle='--', alpha=0.5, label='RTX 5080 (16GB)')
    ax.axhline(y=24, color='orange', linestyle='--', alpha=0.5, label='RTX 4090 (24GB)')
    ax.text(4.2, 16, '16GB VRAM', fontsize=9, color='red')
    ax.text(4.2, 24, '24GB VRAM', fontsize=9, color='orange')
    
    plt.tight_layout()
    plt.show()
    
    print("\nQLoRA 使得在 16GB VRAM 上微調 7B 模型成為可能！")

visualize_qlora_memory()

## Part 5: 訓練資料格式

### 5.1 常見格式

In [ ]:
# 訓練資料格式範例

# 格式 1: Alpaca 格式
alpaca_example = {
    "instruction": "將以下句子翻譯成英文",
    "input": "深度學習是人工智慧的一個重要分支。",
    "output": "Deep learning is an important branch of artificial intelligence."
}

# 格式 2: ShareGPT 格式（對話）
sharegpt_example = {
    "conversations": [
        {"from": "human", "value": "什麼是機器學習？"},
        {"from": "gpt", "value": "機器學習是人工智慧的一個分支..."}
    ]
}

# 格式 3: Instruction 格式
instruction_example = {
    "text": """### Instruction:
將以下句子翻譯成英文

### Input:
深度學習是人工智慧的一個重要分支。

### Response:
Deep learning is an important branch of artificial intelligence."""
}

print("=== Alpaca 格式 ===")
print(json.dumps(alpaca_example, ensure_ascii=False, indent=2))

print("\n=== ShareGPT 格式 ===")
print(json.dumps(sharegpt_example, ensure_ascii=False, indent=2))

print("\n=== Instruction 格式 ===")
print(instruction_example["text"])

In [ ]:
# 資料格式轉換器
class DataFormatter:
    """訓練資料格式化器"""
    
    @staticmethod
    def alpaca_to_text(example: dict, 
                       instruction_template: str = "### Instruction:\n{instruction}\n\n",
                       input_template: str = "### Input:\n{input}\n\n",
                       response_template: str = "### Response:\n{output}") -> str:
        """將 Alpaca 格式轉換為文本"""
        text = instruction_template.format(instruction=example['instruction'])
        
        if example.get('input'):
            text += input_template.format(input=example['input'])
        
        text += response_template.format(output=example['output'])
        return text
    
    @staticmethod
    def sharegpt_to_text(example: dict,
                         human_template: str = "User: {message}\n",
                         assistant_template: str = "Assistant: {message}\n") -> str:
        """將 ShareGPT 格式轉換為文本"""
        text = ""
        for turn in example['conversations']:
            if turn['from'] == 'human':
                text += human_template.format(message=turn['value'])
            else:
                text += assistant_template.format(message=turn['value'])
        return text.strip()
    
    @staticmethod
    def apply_chat_template(messages: List[dict], tokenizer) -> str:
        """
        使用 tokenizer 的 chat template（如果支援）
        
        messages 格式:
        [{"role": "user", "content": "..."},
         {"role": "assistant", "content": "..."}]
        """
        if hasattr(tokenizer, 'apply_chat_template'):
            return tokenizer.apply_chat_template(messages, tokenize=False)
        else:
            # 後備方案
            text = ""
            for msg in messages:
                role = msg['role'].capitalize()
                text += f"{role}: {msg['content']}\n"
            return text

# 測試格式化
formatter = DataFormatter()

print("=== Alpaca 轉換後 ===")
print(formatter.alpaca_to_text(alpaca_example))

print("\n=== ShareGPT 轉換後 ===")
print(formatter.sharegpt_to_text(sharegpt_example))

## Part 6: 使用 PEFT 庫微調

### 6.1 完整微調流程

In [ ]:
# 完整的 LoRA 微調流程（使用 PEFT）
def setup_lora_training():
    """設置 LoRA 微調"""
    try:
        from transformers import (
            AutoModelForCausalLM, 
            AutoTokenizer,
            TrainingArguments,
            Trainer
        )
        from peft import (
            LoraConfig, 
            get_peft_model, 
            TaskType,
            prepare_model_for_kbit_training
        )
        
        print("✓ PEFT 和 Transformers 已安裝")
        return True
    except ImportError as e:
        print(f"✗ 缺少套件: {e}")
        print("請安裝: pip install transformers peft bitsandbytes")
        return False

peft_available = setup_lora_training()

In [ ]:
# LoRA 微調範例（使用小型模型 GPT-2）
if peft_available:
    from transformers import AutoModelForCausalLM, AutoTokenizer
    from peft import LoraConfig, get_peft_model, TaskType
    
    # 載入基礎模型
    model_name = "gpt2"  # 小型模型，適合示範
    print(f"載入模型: {model_name}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # 設定 padding token
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id
    
    # LoRA 配置
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=8,                      # LoRA rank
        lora_alpha=16,            # Alpha 參數
        lora_dropout=0.1,         # Dropout
        target_modules=["c_attn", "c_proj"],  # GPT-2 的注意力層
        bias="none"
    )
    
    # 應用 LoRA
    peft_model = get_peft_model(model, lora_config)
    
    # 打印可訓練參數
    peft_model.print_trainable_parameters()
    
    print(f"\nLoRA 配置:")
    print(f"  Rank: {lora_config.r}")
    print(f"  Alpha: {lora_config.lora_alpha}")
    print(f"  Target modules: {lora_config.target_modules}")

In [ ]:
# 準備訓練資料
if peft_available:
    from torch.utils.data import Dataset
    
    class InstructionDataset(Dataset):
        """指令微調資料集"""
        
        def __init__(self, data: List[dict], tokenizer, max_length: int = 512):
            self.tokenizer = tokenizer
            self.max_length = max_length
            self.data = data
        
        def __len__(self):
            return len(self.data)
        
        def __getitem__(self, idx):
            example = self.data[idx]
            
            # 格式化文本
            text = DataFormatter.alpaca_to_text(example)
            
            # Tokenize
            encodings = self.tokenizer(
                text,
                truncation=True,
                max_length=self.max_length,
                padding="max_length",
                return_tensors="pt"
            )
            
            input_ids = encodings['input_ids'].squeeze()
            attention_mask = encodings['attention_mask'].squeeze()
            
            # 對於 Causal LM，labels = input_ids
            labels = input_ids.clone()
            # 將 padding 位置的 label 設為 -100（不計算損失）
            labels[attention_mask == 0] = -100
            
            return {
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'labels': labels
            }
    
    # 建立示範資料集
    sample_data = [
        {"instruction": "Translate to English", "input": "你好", "output": "Hello"},
        {"instruction": "Translate to English", "input": "謝謝", "output": "Thank you"},
        {"instruction": "What is the capital of France?", "input": "", "output": "Paris"},
        {"instruction": "Summarize this text", "input": "Machine learning is...", "output": "ML is AI that learns from data."},
    ]
    
    dataset = InstructionDataset(sample_data, tokenizer, max_length=128)
    
    print(f"資料集大小: {len(dataset)}")
    
    # 查看一個樣本
    sample = dataset[0]
    print(f"\n樣本 input_ids 形狀: {sample['input_ids'].shape}")
    print(f"解碼後: {tokenizer.decode(sample['input_ids'], skip_special_tokens=True)[:200]}...")

In [ ]:
# 訓練配置（示範）
training_config = {
    "output_dir": "./lora-output",
    "num_train_epochs": 3,
    "per_device_train_batch_size": 4,
    "gradient_accumulation_steps": 4,
    "learning_rate": 2e-4,
    "warmup_steps": 100,
    "logging_steps": 10,
    "save_steps": 500,
    "fp16": True if torch.cuda.is_available() else False,
    "optim": "adamw_torch",
}

print("訓練配置:")
for key, value in training_config.items():
    print(f"  {key}: {value}")

print("\n注意: 實際訓練需要更多資料和計算資源")

## Part 7: 練習題

### Exercise 1: 實作不同 rank 的 LoRA 比較

In [ ]:
def compare_lora_ranks(model_dim: int = 768):
    """
    比較不同 LoRA rank 的效果
    
    測量:
    1. 參數量
    2. 前向傳播時間
    3. 近似誤差（與 full rank 比較）
    """
    ranks = [1, 2, 4, 8, 16, 32, 64]
    results = []
    
    # 建立測試輸入
    x = torch.randn(1, 128, model_dim)
    
    for rank in ranks:
        # 建立 LoRA 層
        lora = LoRALayer(model_dim, model_dim, rank=rank, alpha=rank*2)
        
        # 測量參數量
        trainable_params = lora.trainable_params
        
        # 測量時間
        import time
        start = time.time()
        for _ in range(100):
            _ = lora(x)
        elapsed = (time.time() - start) / 100 * 1000  # ms
        
        results.append({
            'rank': rank,
            'trainable_params': trainable_params,
            'param_ratio': trainable_params / (model_dim * model_dim) * 100,
            'forward_time_ms': elapsed
        })
    
    # 視覺化
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # 參數比例
    ax1.bar([str(r['rank']) for r in results], 
           [r['param_ratio'] for r in results],
           color='steelblue')
    ax1.set_xlabel('LoRA Rank')
    ax1.set_ylabel('Trainable Parameter Ratio (%)')
    ax1.set_title('LoRA Parameter Efficiency')
    ax1.grid(axis='y', alpha=0.3)
    
    # 前向時間
    ax2.plot([r['rank'] for r in results],
            [r['forward_time_ms'] for r in results],
            'o-', color='coral')
    ax2.set_xlabel('LoRA Rank')
    ax2.set_ylabel('Forward Time (ms)')
    ax2.set_title('LoRA Forward Pass Time')
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return results

results = compare_lora_ranks()

print("\n結果摘要:")
for r in results:
    print(f"  Rank {r['rank']:2d}: {r['param_ratio']:.2f}% params, {r['forward_time_ms']:.3f}ms forward")

### Exercise 2: 實作簡單的訓練循環

In [ ]:
def simple_lora_training_loop():
    """
    簡單的 LoRA 訓練循環示範
    """
    if not peft_available:
        print("需要安裝 PEFT")
        return
    
    from torch.utils.data import DataLoader
    
    # 設定
    batch_size = 2
    learning_rate = 1e-4
    num_epochs = 2
    
    # 建立 DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # 只優化 LoRA 參數
    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, peft_model.parameters()),
        lr=learning_rate
    )
    
    # 移到設備
    peft_model.to(device)
    peft_model.train()
    
    losses = []
    
    print(f"開始訓練 (epochs={num_epochs}, batch_size={batch_size}, lr={learning_rate})")
    print("-" * 50)
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch_idx, batch in enumerate(dataloader):
            # 移到設備
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # 前向傳播
            outputs = peft_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss
            
            # 反向傳播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            losses.append(loss.item())
        
        avg_loss = epoch_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")
    
    # 視覺化損失
    plt.figure(figsize=(10, 4))
    plt.plot(losses, 'b-', alpha=0.7)
    plt.xlabel('Step')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.grid(alpha=0.3)
    plt.show()
    
    return losses

if peft_available:
    losses = simple_lora_training_loop()

## 總結

```
┌─────────────────────────────────────────────────────────────┐
│                   LLM 微調技術總結                           │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  1. 微調動機                                                │
│     • 領域適應、任務特化、風格調整                           │
│     • Full FT 記憶體需求過高                                │
│                                                             │
│  2. PEFT 方法                                               │
│     • LoRA: 低秩分解權重更新                                │
│     • QLoRA: 4-bit 量化 + LoRA                              │
│     • Adapter, Prefix Tuning 等                             │
│                                                             │
│  3. LoRA 關鍵參數                                           │
│     • rank (r): 控制可訓練參數量                            │
│     • alpha: 縮放因子                                       │
│     • target_modules: 應用 LoRA 的層                        │
│                                                             │
│  4. 訓練資料                                                │
│     • Alpaca 格式                                           │
│     • ShareGPT 格式                                         │
│     • Chat template                                         │
│                                                             │
│  5. 實用建議                                                │
│     • rank=8-32 通常足夠                                    │
│     • 優先對 Q, V 投影應用 LoRA                             │
│     • 使用 QLoRA 在消費級 GPU 上訓練                        │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### 下一步學習

- **災難性遺忘**: `transfer_adaptation/catastrophic_forgetting.ipynb`
- **RLHF**: `reinforcement_learning/rlhf_alignment.ipynb`
- **模型合併**: `llm_advanced/model_merging.ipynb`

## 參考資源

### 課程
- [李宏毅 2025 Spring ML HW5](https://speech.ee.ntu.edu.tw/~hylee/ml/2025-spring.php)
- [李宏毅 2025 Fall GenAI-ML HW7](https://speech.ee.ntu.edu.tw/~hylee/GenAI-ML/2025-fall.php)

### 論文
- [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)
- [QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314)

### 工具
- [PEFT (Hugging Face)](https://github.com/huggingface/peft)
- [bitsandbytes](https://github.com/TimDettmers/bitsandbytes)